In [1]:
import librosa
from librosa import display
import matplotlib.pyplot as plt
import scipy
import os
import tensorflow as tf
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dropout, Dense
from keras.losses import binary_crossentropy,categorical_crossentropy
from keras.activations import sigmoid, relu
from keras.models import Sequential
from keras.optimizers import rmsprop, Adam
from keras.preprocessing.image import ImageDataGenerator
import csv
import pandas as pd
import numpy as np
from numpy import array
import PIL

Using TensorFlow backend.


In [2]:
%config IPCompleter.greedy=True
#try optimizer adam instead of rmsprop

# Preprocessing of Data


In [3]:
path_directory_training= r'C:\Users\Gebruiker\Desktop\MonoPhonic\data\training\images'
path_labels_training= r'C:\Users\Gebruiker\Desktop\MonoPhonic\data\training\labels.csv'

path_labels_valid =  r'C:\Users\Gebruiker\Desktop\MonoPhonic\data\valid\labels.csv'
path_directory_valid =  r'C:\Users\Gebruiker\Desktop\MonoPhonic\data\valid\images'

path_directory_test = r'C:\Users\Gebruiker\Desktop\MonoPhonic\data\test\images'
path_labels_test =   r'C:\Users\Gebruiker\Desktop\MonoPhonic\data\test\labels.csv'

class_list = ['A0','A#0','B0','C1','C#1','D1','D#1','E1','F1',
         'F#1','G1','G#1','A1','A#1','B1','C2','C#2','D2','D#2','E2','F2',
         'F#2','G2','G#2','A2','A#2','B2','C3','C#3','D3','D#3','E3','F3',
         'F#3','G3','G#3','A3','A#3','B3','C4','C#4','D4','D#4','E4','F4',
         'F#4','G4','G#4','A4','A#4','B4','C5','C#5','D5','D#5','E5','F5',
         'F#5','G5','G#5','A5','A#5','B5','C6','C#6','D6','D#6','E6','F6',
         'F#6','G6','G#6','A6','A#6','B6','C7','C#7','D7','D#7','E7','F7',
         'F#7','G7','G#7','A7','A#7','B7','C8']



In [4]:
df_training = pd.read_csv(path_labels_training)

df_valid = pd.read_csv(path_labels_valid)

df_test = pd.read_csv(path_labels_test)

datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_dataframe(
df_training[0:32],
directory=path_directory_training,
x_col='filename',
y_col=class_list,
batch_size=32,
target_size=(100,100),
class_mode='raw',
shuffle= False)

validation_generator = datagen.flow_from_dataframe(
df_valid,
directory=path_directory_valid,
x_col='filename',
y_col=class_list,
batch_size=32,
target_size=(100,100),
class_mode='raw',
shuffle= True,
color_mode='grayscale')


datagenTest = ImageDataGenerator()
test_generator = datagenTest.flow_from_dataframe(
df_test,
class_mode = None,
directory = path_directory_test,
color_mode='grayscale',
target_size=(100,100),
shuffle= True,
batch_size = 32)

training_steps = train_generator.n // train_generator.batch_size
validation_steps = validation_generator.n // validation_generator.batch_size
test_steps = test_generator.n // test_generator.batch_size


Found 32 validated image filenames.
Found 2080 validated image filenames.
Found 651 validated image filenames.


In [11]:
samples,labels = next(train_generator)

In [14]:
samples[0].shape

(100, 100, 3)

# Building of model

In [3]:
model = Sequential()

model.add(Conv2D(64,(3,3), activation='relu',input_shape=(100,100,1)))
model.add(Dropout(0.5))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.5))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(88, activation='sigmoid'))


In [ ]:
model.compile(loss=binary_crossentropy,metrics=['accuracy'],optimizer='Adam')


# Training and evaluation of model

In [ ]:
history = model.fit_generator(generator=train_generator,
                    steps_per_epoch=training_steps,
                    epochs=2,validation_data=validation_generator,
                    validation_steps=validation_steps)

model.save( r'C:\Users\Gebruiker\Desktop\model.h5')

In [ ]:
def generateSpectogram(sourcePath,targetPath):     #generate Mel-spectogram from wav 
    plt.box(0)
    base, ext= os.path.splitext(file) #saving generated spectogram in target directory
    samples, sampling_rate = librosa.load(sourcePath)
    spectogram = librosa.stft(samples)
    mel = librosa.feature.melspectrogram(y=samples,sr=sampling_rate,S=spectogram,n_fft=2048,hop_length=512,n_mels=128)
    mel_dB = librosa.amplitude_to_db(np.abs(mel)) 
    librosa.display.specshow(mel_dB,cmap='gray')
    plt.savefig(targetPath  + f"\\{base}" + ".png")
    plt.close()

In [ ]:

for file in os.listdir('C:\\Users\\Gebruiker\\Desktop\\testSamples-Guitar\\'):
    generateSpectogram('C:\\Users\\Gebruiker\\Desktop\\testSamples-Guitar\\'+ f"\\{file}",'C:\\Users\\Gebruiker\\Desktop\\testSamples-Guitar\\New folder')  #generating spectogram


In [1]:
class_list = ['A0','A#0','B0','C1','C#1','D1','D#1','E1','F1',
         'F#1','G1','G#1','A1','A#1','B1','C2','C#2','D2','D#2','E2','F2',
         'F#2','G2','G#2','A2','A#2','B2','C3','C#3','D3','D#3','E3','F3',
         'F#3','G3','G#3','A3','A#3','B3','C4','C#4','D4','D#4','E4','F4',
         'F#4','G4','G#4','A4','A#4','B4','C5','C#5','D5','D#5','E5','F5',
         'F#5','G5','G#5','A5','A#5','B5','C6','C#6','D6','D#6','E6','F6',
         'F#6','G6','G#6','A6','A#6','B6','C7','C#7','D7','D#7','E7','F7',
         'F#7','G7','G#7','A7','A#7','B7','C8']

noteDictionary = {}

for index,note in enumerate(class_list):
    noteDictionary[index] = note




In [ ]:
test_generator.reset
model = tf.keras.models.load_model( r'C:\Users\Gebruiker\Desktop\model.h5')
#predictions = model.predict_generator(test_generator,steps = test_steps)

predictions = predictions.tolist()
for i,prediction in enumerate(predictions):
    print(i)
    try:
        indexOutput = prediction.index(1)
    except:
        print('Could not generate output for this image')
    else:
        print(noteDictionary.get(indexOutput))
 

In [2]:
class_list

['A0',
 'A#0',
 'B0',
 'C1',
 'C#1',
 'D1',
 'D#1',
 'E1',
 'F1',
 'F#1',
 'G1',
 'G#1',
 'A1',
 'A#1',
 'B1',
 'C2',
 'C#2',
 'D2',
 'D#2',
 'E2',
 'F2',
 'F#2',
 'G2',
 'G#2',
 'A2',
 'A#2',
 'B2',
 'C3',
 'C#3',
 'D3',
 'D#3',
 'E3',
 'F3',
 'F#3',
 'G3',
 'G#3',
 'A3',
 'A#3',
 'B3',
 'C4',
 'C#4',
 'D4',
 'D#4',
 'E4',
 'F4',
 'F#4',
 'G4',
 'G#4',
 'A4',
 'A#4',
 'B4',
 'C5',
 'C#5',
 'D5',
 'D#5',
 'E5',
 'F5',
 'F#5',
 'G5',
 'G#5',
 'A5',
 'A#5',
 'B5',
 'C6',
 'C#6',
 'D6',
 'D#6',
 'E6',
 'F6',
 'F#6',
 'G6',
 'G#6',
 'A6',
 'A#6',
 'B6',
 'C7',
 'C#7',
 'D7',
 'D#7',
 'E7',
 'F7',
 'F#7',
 'G7',
 'G#7',
 'A7',
 'A#7',
 'B7',
 'C8']